# Step 3: Embeddings & Positional Encoding 📍

In this notebook, we'll understand how **token embeddings** and **positional embeddings** work together to give the transformer a rich representation of input text.

## What We'll Learn

1. **Why Embeddings?** - Why we can't just use token IDs directly
2. **Token Embeddings** - Convert token IDs → dense vectors
3. **Position Embeddings** - Encode where each token appears
4. **Combining Embeddings** - The input to the transformer
5. **Visualizing Embeddings** - See semantic relationships
6. **Similarity Search** - Find related tokens using embeddings

---

## The Big Picture: From Text to Vectors

We've already seen how tokenization converts text to token IDs:

```
"Hello world" → [15496, 995]
```

But the transformer can't work with raw IDs! It needs **dense vectors** that capture meaning:

```
"Hello world" → [15496, 995] → [[0.12, -0.34, ...], [-0.05, 0.78, ...]] → Transformer
                 token IDs        768-dim embeddings (2 × 768 matrix)
```

This is what embeddings do!


---

## Setup: Load Model Weights and Tokenizer

We'll reuse what we built in Steps 1 and 2.


In [103]:
import json
import numpy as np
import regex
from functools import lru_cache


In [104]:
# Load SafeTensors (from Step 1)
def load_safetensors(path: str) -> dict:
    """Load a SafeTensors file without external library."""
    with open(path, 'rb') as f:
        header_size = int.from_bytes(f.read(8), 'little')
        header_json = f.read(header_size).decode('utf-8')
        header = json.loads(header_json)
        header.pop("__metadata__", {})
        
        tensors = {}
        data_start = 8 + header_size
        
        for name, info in header.items():
            dtype_map = {"F32": np.float32, "F16": np.float16, "I64": np.int64}
            dtype = dtype_map.get(info["dtype"], np.float32)
            offset_start, offset_end = info["data_offsets"]
            
            f.seek(data_start + offset_start)
            raw_data = f.read(offset_end - offset_start)
            tensors[name] = np.frombuffer(raw_data, dtype=dtype).reshape(info["shape"])
        
        return tensors

# Load model weights
tensors = load_safetensors("models/gpt2/model.safetensors")

# Load config
with open("models/gpt2/config.json") as f:
    config = json.load(f)

print(f"✅ Loaded {len(tensors)} weight tensors")
print(f"\nKey hyperparameters:")
print(f"  n_vocab: {config['vocab_size']:,}")
print(f"  n_embd:  {config['n_embd']}")
print(f"  n_ctx:   {config['n_ctx']}")


✅ Loaded 160 weight tensors

Key hyperparameters:
  n_vocab: 50,257
  n_embd:  768
  n_ctx:   1024


In [105]:
# GPT-2 Tokenizer (from Step 2)
def bytes_to_unicode():
    """Create byte-to-unicode mapping for GPT-2."""
    printable = list(range(ord("!"), ord("~") + 1))
    printable += list(range(ord("¡"), ord("¬") + 1))
    printable += list(range(ord("®"), ord("ÿ") + 1))
    
    byte_values = printable[:]
    unicode_chars = printable[:]
    
    n = 0
    for b in range(256):
        if b not in printable:
            byte_values.append(b)
            unicode_chars.append(256 + n)
            n += 1
    
    return {b: chr(c) for b, c in zip(byte_values, unicode_chars)}


class GPT2Tokenizer:
    """BPE tokenizer for GPT-2 (from Step 2)."""
    
    def __init__(self, vocab_path="models/gpt2/vocab.json", merges_path="models/gpt2/merges.txt"):
        with open(vocab_path, "r", encoding="utf-8") as f:
            self.vocab = json.load(f)
        self.vocab_inverse = {v: k for k, v in self.vocab.items()}
        
        with open(merges_path, "r", encoding="utf-8") as f:
            merges_text = f.read()
        merge_lines = [line for line in merges_text.split("\n")[1:] if line]
        self.bpe_merges = {}
        for i, line in enumerate(merge_lines):
            parts = line.split(" ")
            if len(parts) == 2:
                self.bpe_merges[tuple(parts)] = i
        
        self.byte_encoder = bytes_to_unicode()
        self.byte_decoder = {v: k for k, v in self.byte_encoder.items()}
        self.pattern = regex.compile(
            r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        )
        self.cache = {}
    
    def _get_pairs(self, word):
        return set((word[i], word[i + 1]) for i in range(len(word) - 1))
    
    def _bpe(self, token):
        if token in self.cache:
            return self.cache[token]
        
        word = list(token)
        while len(word) > 1:
            pairs = self._get_pairs(word)
            if not pairs:
                break
            best_pair = min(pairs, key=lambda p: self.bpe_merges.get(p, float('inf')))
            if best_pair not in self.bpe_merges:
                break
            first, second = best_pair
            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and word[i] == first and word[i + 1] == second:
                    new_word.append(first + second)
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            word = new_word
        
        result = tuple(word)
        self.cache[token] = result
        return result
    
    def encode(self, text):
        token_ids = []
        for chunk in self.pattern.findall(text):
            byte_encoded = ''.join(self.byte_encoder[b] for b in chunk.encode('utf-8'))
            bpe_tokens = self._bpe(byte_encoded)
            for token in bpe_tokens:
                if token in self.vocab:
                    token_ids.append(self.vocab[token])
        return token_ids
    
    def decode(self, token_ids):
        text = ''.join(self.vocab_inverse[tid] for tid in token_ids)
        byte_values = [self.byte_decoder[c] for c in text]
        return bytes(byte_values).decode('utf-8', errors='replace')
    
    def get_token_string(self, token_id):
        """Get the string representation of a token ID."""
        return self.vocab_inverse.get(token_id, "<UNK>")


# Create tokenizer
tokenizer = GPT2Tokenizer()
print(f"✅ Tokenizer loaded with {len(tokenizer.vocab):,} tokens")


✅ Tokenizer loaded with 50,257 tokens


---

## Why Can't We Just Use Token IDs?

You might wonder: why not just feed the token IDs `[15496, 995]` directly into the neural network?

### Problem 1: IDs Don't Capture Meaning

Token IDs are **arbitrary numbers**. The ID assignment has no semantic meaning:

```
"king" → ID 3364
"queen" → ID 21901
"banana" → ID 39897
```

Mathematically, `3364` is closer to `21901` than to `39897`, but that says nothing about meaning! We need a representation where **similar words are close together**.

### Problem 2: Discrete vs. Continuous

Neural networks work best with **continuous** values. Token IDs are discrete — there's no "in between" token 3364 and 3365. Embeddings give us a continuous space where:

- Small changes in the vector → small changes in meaning
- We can do arithmetic (famous: `king - man + woman ≈ queen`)
- Gradients can flow smoothly during training

### Problem 3: Dimensionality

A single token ID is 1 number. The model needs **768 dimensions** of features to work with! Each dimension can capture different aspects of meaning:

- Is this a noun or verb?
- Is this formal or casual?
- Is this about animals, food, technology?
- Is this positive or negative?


---

## Token Embeddings: `wte.weight`

The **token embedding matrix** is simply a lookup table:

```
wte.weight: [50257, 768]
            vocab size   embedding dim
```

Each of the 50,257 tokens has its own 768-dimensional vector. To embed a token, we just **look up its row**:

```python
token_id = 15496  # "Hello"
embedding = wte.weight[token_id]  # 768-dim vector
```


In [106]:
# Get the token embedding matrix
wte = tensors["wte.weight"]

print("Token Embedding Matrix (wte.weight)")
print(f"  Shape: {wte.shape}")
print(f"  - {wte.shape[0]:,} tokens (vocabulary size)")
print(f"  - {wte.shape[1]} dimensions per token")
print(f"  Total parameters: {wte.size:,}")
print(f"  Memory: {wte.nbytes / 1e6:.1f} MB")


Token Embedding Matrix (wte.weight)
  Shape: (50257, 768)
  - 50,257 tokens (vocabulary size)
  - 768 dimensions per token
  Total parameters: 38,597,376
  Memory: 154.4 MB


In [107]:
def get_token_embedding(token_id):
    """
    Look up the embedding for a token ID.
    
    This is the fundamental operation: token_id → 768-dim vector
    """
    return wte[token_id]


# Let's embed the word "Hello"
hello_ids = tokenizer.encode("Hello")
print(f"'Hello' → Token ID: {hello_ids[0]}")

embedding = get_token_embedding(hello_ids[0])
print(f"Embedding shape: {embedding.shape}")
print(f"\nFirst 20 values of the embedding:")
print(embedding[:20])


'Hello' → Token ID: 15496
Embedding shape: (768,)

First 20 values of the embedding:
[-0.06865076 -0.1326938   0.0112033  -0.14665768 -0.1841705  -0.03579693
 -0.21733314 -0.17133704 -0.01038615 -0.01419895  0.00569266 -0.12377474
 -0.00720884  0.04176147 -0.03684433 -0.09803814  0.13315281  0.0523295
 -0.15724912  0.11166326]


In [108]:
# Let's look at embeddings for a few words
words_to_embed = ["Hello", " the", " king", " queen", " man", " woman", " dog", " cat"]

print("Token Embeddings for Different Words")
print("=" * 70)
print(f"{'Word':<12} {'Token ID':>10} {'First 5 values of embedding'}")
print("-" * 70)

for word in words_to_embed:
    token_ids = tokenizer.encode(word)
    if len(token_ids) == 1:  # Only show single-token words
        token_id = token_ids[0]
        emb = get_token_embedding(token_id)
        values_str = ", ".join(f"{v:.3f}" for v in emb[:5])
        print(f"'{word}'  {token_id:>10}   [{values_str}, ...]")


Token Embeddings for Different Words
Word           Token ID First 5 values of embedding
----------------------------------------------------------------------
'Hello'       15496   [-0.069, -0.133, 0.011, -0.147, -0.184, ...]
' the'         262   [-0.039, 0.005, 0.042, 0.040, -0.036, ...]
' king'        5822   [-0.019, -0.117, 0.089, 0.073, 0.165, ...]
' queen'       16599   [-0.020, -0.290, -0.032, -0.059, 0.052, ...]
' man'         582   [0.004, -0.005, 0.010, -0.033, -0.054, ...]
' woman'        2415   [0.040, -0.031, -0.028, -0.043, -0.148, ...]
' dog'        3290   [0.094, -0.077, 0.035, -0.035, -0.089, ...]
' cat'        3797   [0.010, 0.037, 0.164, -0.219, 0.029, ...]


---

## Embedding a Full Sentence

When we embed a sentence, we look up the embedding for **each token** and stack them:

```
"Hello world" → [15496, 995] → [[emb_0], [emb_1]] → shape: (2, 768)
```

The result is a 2D matrix where:
- Each **row** is a token's embedding
- Each **column** is a feature dimension


In [109]:
def embed_tokens(token_ids):
    """
    Embed a sequence of token IDs.
    
    Args:
        token_ids: List of token IDs
        
    Returns:
        numpy array of shape (seq_len, 768)
    """
    # This is just a batch lookup: wte[token_ids]
    return wte[token_ids]


# Embed a sentence
sentence = "Hello world"
token_ids = tokenizer.encode(sentence)

print(f"Sentence: '{sentence}'")
print(f"Token IDs: {token_ids}")

# Get embeddings
token_embeddings = embed_tokens(token_ids)

print(f"\nToken Embeddings shape: {token_embeddings.shape}")
print(f"  - {token_embeddings.shape[0]} tokens")
print(f"  - {token_embeddings.shape[1]} dimensions each")


Sentence: 'Hello world'
Token IDs: [15496, 995]

Token Embeddings shape: (2, 768)
  - 2 tokens
  - 768 dimensions each


In [110]:
# Visualize the embedding matrix for the sentence
print(f"\nEmbedding matrix for '{sentence}':")
print()

# Show first 10 dimensions for each token
print("First 10 dimensions of each token's embedding:")
print()
print(f"{'Token':<15} {'Dim 0':>8} {'Dim 1':>8} {'Dim 2':>8} {'Dim 3':>8} {'Dim 4':>8} ...")
print("-" * 70)

for i, tid in enumerate(token_ids):
    token_str = tokenizer.get_token_string(tid).replace("Ġ", "⎵")
    emb = token_embeddings[i]
    values = " ".join(f"{v:>8.3f}" for v in emb[:5])
    print(f"{token_str:<15} {values} ...")



Embedding matrix for 'Hello world':

First 10 dimensions of each token's embedding:

Token              Dim 0    Dim 1    Dim 2    Dim 3    Dim 4 ...
----------------------------------------------------------------------
Hello             -0.069   -0.133    0.011   -0.147   -0.184 ...
⎵world            -0.149    0.152    0.006   -0.114   -0.026 ...


---

## Position Embeddings: `wpe.weight`

Transformers process all tokens **in parallel** (unlike RNNs which process left-to-right). Without position information, the model can't tell the difference between:

```
"The cat sat on the mat"
"mat the on sat cat The"
```

Both would produce the same set of token embeddings (just in a different order), but the model wouldn't know the order!

### GPT-2's Solution: Learned Position Embeddings

GPT-2 has a separate embedding table for **positions**:

```
wpe.weight: [1024, 768]
            max positions  embedding dim
```

Each position (0, 1, 2, ..., 1023) has its own learned 768-dimensional vector.


In [111]:
# Get the position embedding matrix
wpe = tensors["wpe.weight"]

print("Position Embedding Matrix (wpe.weight)")
print(f"  Shape: {wpe.shape}")
print(f"  - {wpe.shape[0]:,} positions (max context length)")
print(f"  - {wpe.shape[1]} dimensions per position")
print(f"  Total parameters: {wpe.size:,}")
print(f"  Memory: {wpe.nbytes / 1e6:.1f} MB")


Position Embedding Matrix (wpe.weight)
  Shape: (1024, 768)
  - 1,024 positions (max context length)
  - 768 dimensions per position
  Total parameters: 786,432
  Memory: 3.1 MB


In [112]:
def get_position_embedding(position):
    """
    Look up the embedding for a position.
    
    Args:
        position: Integer position (0 to 1023)
        
    Returns:
        768-dim position embedding vector
    """
    if position >= wpe.shape[0]:
        raise ValueError(f"Position {position} exceeds max context length {wpe.shape[0]}")
    return wpe[position]


# Compare position embeddings
print("Position Embeddings Comparison")
print("=" * 60)

positions = [0, 1, 2, 100, 500, 1023]
for pos in positions:
    emb = get_position_embedding(pos)
    values_str = ", ".join(f"{v:.3f}" for v in emb[:5])
    print(f"Position {pos:4}: [{values_str}, ...]")


Position Embeddings Comparison
Position    0: [-0.019, -0.197, 0.004, 0.011, 0.064, ...]
Position    1: [0.024, -0.054, -0.095, -0.013, -0.010, ...]
Position    2: [0.004, -0.085, 0.055, -0.005, -0.026, ...]
Position  100: [-0.000, -0.016, -0.010, -0.002, 0.001, ...]
Position  500: [-0.003, 0.004, -0.029, 0.004, 0.003, ...]
Position 1023: [0.000, 0.003, -0.002, 0.003, -0.001, ...]


In [113]:
def embed_positions(seq_len):
    """
    Get position embeddings for a sequence.
    
    Args:
        seq_len: Length of the sequence
        
    Returns:
        numpy array of shape (seq_len, 768)
    """
    positions = np.arange(seq_len)
    return wpe[positions]


# Get position embeddings for our sentence
seq_len = len(token_ids)  # From "Hello world"
position_embeddings = embed_positions(seq_len)

print(f"Position Embeddings for sequence length {seq_len}:")
print(f"  Shape: {position_embeddings.shape}")


Position Embeddings for sequence length 2:
  Shape: (2, 768)


---

## Why Learned Positions (vs. Fixed Sinusoidal)?

The original Transformer paper ("Attention Is All You Need") used **fixed sinusoidal** position encodings:

```
PE(pos, 2i) = sin(pos / 10000^(2i/d))
PE(pos, 2i+1) = cos(pos / 10000^(2i/d))
```

GPT-2 instead uses **learned** position embeddings — the model discovers what position patterns are useful during training.

| Approach | Pros | Cons |
|----------|------|------|
| Sinusoidal | Generalizes to longer sequences, no extra parameters | Fixed patterns may not be optimal |
| Learned | Model can learn optimal patterns | Limited to training context length |

**Note:** Modern models like LLaMA, GPT-4 use **RoPE** (Rotary Position Embedding) which combines benefits of both!


---

## Combining Token + Position Embeddings

The input to the transformer is simply the **sum** of token and position embeddings:

```
input_embedding = token_embedding + position_embedding
```

This might seem strange — why addition? Why not concatenation?

### Why Addition Works

1. **Same dimensionality**: Both are 768-dim, so addition is natural
2. **Information superposition**: The model learns to encode different info in different dimensions
3. **Efficiency**: No extra parameters or dimension increase
4. **It works empirically!**: The model learns to disentangle meaning from position

Think of it like mixing colors — red + blue = purple, but you can still tell both were there.


In [114]:
def get_input_embeddings(text):
    """
    Get the full input embeddings for text.
    
    This is the first step in GPT-2's forward pass:
    1. Tokenize text -> token IDs
    2. Look up token embeddings
    3. Look up position embeddings
    4. Add them together
    
    Args:
        text: Input string
        
    Returns:
        tuple: (input_embeddings, token_ids, tokens)
    """
    # Step 1: Tokenize
    token_ids = tokenizer.encode(text)
    seq_len = len(token_ids)
    
    # Step 2: Token embeddings (look up wte)
    token_emb = wte[token_ids]  # Shape: (seq_len, 768)
    
    # Step 3: Position embeddings (look up wpe)
    positions = np.arange(seq_len)
    position_emb = wpe[positions]  # Shape: (seq_len, 768)
    
    # Step 4: Add them!
    input_emb = token_emb + position_emb  # Shape: (seq_len, 768)
    
    # Get token strings for display
    tokens = [tokenizer.get_token_string(tid) for tid in token_ids]
    
    return input_emb, token_ids, tokens


In [115]:
# Let's see the full embedding process
text = "The quick brown fox"

print(f"Embedding the text: '{text}'")
print("=" * 60)

input_emb, token_ids, tokens = get_input_embeddings(text)

print(f"\n1. Tokenization:")
for i, (tid, tok) in enumerate(zip(token_ids, tokens)):
    display = tok.replace("Ġ", "⎵")
    print(f"   Position {i}: '{display}' -> ID {tid}")

print(f"\n2. Token Embeddings: shape {wte[token_ids].shape}")
print(f"3. Position Embeddings: shape {wpe[:len(token_ids)].shape}")
print(f"4. Combined Input: shape {input_emb.shape}")

print(f"\n✅ Ready for the transformer! Input shape: {input_emb.shape}")


Embedding the text: 'The quick brown fox'

1. Tokenization:
   Position 0: 'The' -> ID 464
   Position 1: '⎵quick' -> ID 2068
   Position 2: '⎵brown' -> ID 7586
   Position 3: '⎵fox' -> ID 21831

2. Token Embeddings: shape (4, 768)
3. Position Embeddings: shape (4, 768)
4. Combined Input: shape (4, 768)

✅ Ready for the transformer! Input shape: (4, 768)


In [116]:
# Let's visualize how the embeddings combine
print("\nHow embeddings combine (first 3 dimensions):")
print("=" * 75)
print(f"{'Token':<12} {'Token Emb':>25} + {'Pos Emb':>22} = {'Combined'}")
print("-" * 75)

for i, (tid, tok) in enumerate(zip(token_ids, tokens)):
    tok_emb = wte[tid][:3]
    pos_emb = wpe[i][:3]
    combined = input_emb[i][:3]
    
    display = tok.replace("Ġ", "⎵")
    tok_str = "[" + ", ".join(f"{v:.2f}" for v in tok_emb) + "]"
    pos_str = "[" + ", ".join(f"{v:+.2f}" for v in pos_emb) + "]"
    comb_str = "[" + ", ".join(f"{v:.2f}" for v in combined) + "]"
    
    print(f"{display:<12} {tok_str:>25} + {pos_str:>22} = {comb_str}")



How embeddings combine (first 3 dimensions):
Token                        Token Emb +                Pos Emb = Combined
---------------------------------------------------------------------------
The               [-0.07, -0.02, 0.06] +  [-0.02, -0.20, +0.00] = [-0.09, -0.22, 0.07]
⎵quick            [-0.01, -0.03, 0.20] +  [+0.02, -0.05, -0.09] = [0.02, -0.08, 0.10]
⎵brown             [-0.05, 0.12, 0.08] +  [+0.00, -0.08, +0.05] = [-0.05, 0.03, 0.13]
⎵fox                [0.10, 0.00, 0.11] +  [-0.00, -0.07, +0.11] = [0.10, -0.07, 0.22]


---

## Visualizing Embedding Space

Embeddings live in a 768-dimensional space. We can't visualize 768 dimensions, but we can use **cosine similarity** to measure how close embeddings are.

Let's explore:
1. **Token similarity** - Similar words should be close together
2. **Position patterns** - How position embeddings change across the sequence


In [117]:
def cosine_similarity(a, b):
    """
    Compute cosine similarity between two vectors.
    
    Cosine similarity measures the angle between vectors:
    - 1.0 = identical direction (very similar)
    - 0.0 = perpendicular (unrelated)
    - -1.0 = opposite direction (opposite meaning)
    """
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)


def compare_words(word1, word2):
    """Compare the similarity of two words using their embeddings."""
    ids1 = tokenizer.encode(word1)
    ids2 = tokenizer.encode(word2)
    
    # Only compare single-token words
    if len(ids1) != 1 or len(ids2) != 1:
        return None
    
    emb1 = wte[ids1[0]]
    emb2 = wte[ids2[0]]
    
    return cosine_similarity(emb1, emb2)


# Compare some word pairs
print("Word Similarity using Cosine Similarity")
print("=" * 50)
print("(1.0 = identical, 0.0 = unrelated, -1.0 = opposite)\n")

word_pairs = [
    (" king", " queen"),
    (" king", " man"),
    (" queen", " woman"),
    (" dog", " cat"),
    (" dog", " computer"),
    (" happy", " sad"),
    (" happy", " joyful"),
    (" run", " running"),
]

for w1, w2 in word_pairs:
    sim = compare_words(w1, w2)
    if sim is not None:
        bar = "█" * int(sim * 20) if sim > 0 else ""
        print(f"  {w1.strip():<10} <-> {w2.strip():<10}: {sim:+.3f} {bar}")


Word Similarity using Cosine Similarity
(1.0 = identical, 0.0 = unrelated, -1.0 = opposite)

  king       <-> queen     : +0.657 █████████████
  king       <-> man       : +0.367 ███████
  queen      <-> woman     : +0.451 █████████
  dog        <-> cat       : +0.550 ██████████
  dog        <-> computer  : +0.303 ██████
  happy      <-> sad       : +0.550 ██████████
  happy      <-> joyful    : +0.589 ███████████
  run        <-> running   : +0.630 ████████████


---

## The Famous Word Analogy Test

One of the most fascinating properties of word embeddings is that they capture **semantic relationships** through vector arithmetic:

```
king - man + woman ≈ queen
```

This works because the embedding space encodes:
- `king - man` = "royalty" concept
- Adding `woman` to "royalty" = `queen`

Let's test this with GPT-2's embeddings!


In [118]:
def get_single_token_embedding(word):
    """Get embedding for a single-token word."""
    ids = tokenizer.encode(word)
    if len(ids) != 1:
        raise ValueError(f"'{word}' is not a single token (got {len(ids)} tokens)")
    return wte[ids[0]]


def find_closest_tokens(embedding, top_k=10, exclude_ids=None):
    """
    Find the tokens with embeddings closest to the given embedding.
    
    Args:
        embedding: Target 768-dim vector
        top_k: Number of closest tokens to return
        exclude_ids: Token IDs to exclude from results
        
    Returns:
        List of (token_id, token_string, similarity) tuples
    """
    exclude_ids = exclude_ids or []
    
    # Normalize the query embedding
    embedding_norm = embedding / np.linalg.norm(embedding)
    
    # Normalize all token embeddings
    wte_norms = np.linalg.norm(wte, axis=1, keepdims=True)
    wte_normalized = wte / wte_norms
    
    # Compute similarities (dot product of normalized vectors = cosine similarity)
    similarities = wte_normalized @ embedding_norm
    
    # Get top-k indices
    top_indices = np.argsort(similarities)[::-1]  # Descending order
    
    results = []
    for idx in top_indices:
        if idx not in exclude_ids:
            token_str = tokenizer.get_token_string(idx)
            results.append((idx, token_str, similarities[idx]))
            if len(results) >= top_k:
                break
    
    return results


In [119]:
# Test: king - man + woman = ?
print("Word Analogy: king - man + woman = ?")
print("=" * 50)

try:
    king = get_single_token_embedding(" king")
    man = get_single_token_embedding(" man")
    woman = get_single_token_embedding(" woman")
    
    # Compute: king - man + woman
    result_embedding = king - man + woman
    
    # Find closest tokens (exclude the input words)
    exclude = [tokenizer.encode(w)[0] for w in [" king", " man", " woman"]]
    closest = find_closest_tokens(result_embedding, top_k=10, exclude_ids=exclude)
    
    print("\nTop 10 closest tokens to (king - man + woman):")
    print("-" * 50)
    for i, (tid, token_str, sim) in enumerate(closest, 1):
        display = token_str.replace("Ġ", "⎵")
        highlight = "👑" if "queen" in token_str.lower() else ""
        print(f"  {i:2}. '{display}' (similarity: {sim:.4f}) {highlight}")

except Exception as e:
    print(f"Error: {e}")


Word Analogy: king - man + woman = ?

Top 10 closest tokens to (king - man + woman):
--------------------------------------------------
   1. '⎵queen' (similarity: 0.7085) 👑
   2. '⎵princess' (similarity: 0.6046) 
   3. '⎵Queen' (similarity: 0.5964) 👑
   4. '⎵kings' (similarity: 0.5932) 
   5. 'Queen' (similarity: 0.5720) 👑
   6. '⎵King' (similarity: 0.5390) 
   7. '⎵monarch' (similarity: 0.5248) 
   8. '⎵queens' (similarity: 0.5193) 👑
   9. '⎵goddess' (similarity: 0.5166) 
  10. '⎵ruler' (similarity: 0.5065) 


In [120]:
# Let's try more analogies!
analogies = [
    (" Paris", " France", " Tokyo", "Japan"),      # capital : country
    (" big", " bigger", " small", "smaller"),     # comparative
    (" walk", " walked", " run", "ran"),          # past tense
]

print("\nMore Word Analogies")
print("=" * 60)

for a, b, c, expected in analogies:
    try:
        emb_a = get_single_token_embedding(a)
        emb_b = get_single_token_embedding(b)
        emb_c = get_single_token_embedding(c)
        
        # a is to b as c is to ?
        # result = b - a + c
        result = emb_b - emb_a + emb_c
        
        exclude = [tokenizer.encode(w)[0] for w in [a, b, c]]
        closest = find_closest_tokens(result, top_k=3, exclude_ids=exclude)
        
        print(f"\n{a.strip()} : {b.strip()} :: {c.strip()} : ?")
        print(f"  Expected: {expected}")
        print(f"  Top 3 predictions:")
        for tid, tok, sim in closest:
            display = tok.replace("Ġ", "⎵")
            match = "✓" if expected.lower() in tok.lower() else ""
            print(f"    '{display}' ({sim:.3f}) {match}")
    
    except Exception as e:
        print(f"\n{a} : {b} :: {c} : ? - Error: {e}")



More Word Analogies

Paris : France :: Tokyo : ?
  Expected: Japan
  Top 3 predictions:
    '⎵Japan' (0.723) ✓
    'Japan' (0.658) ✓
    '⎵Japanese' (0.559) ✓

big : bigger :: small : ?
  Expected: smaller
  Top 3 predictions:
    '⎵smaller' (0.797) ✓
    '⎵larger' (0.762) 
    'small' (0.608) 

walk : walked :: run : ?
  Expected: ran
  Top 3 predictions:
    '⎵ran' (0.742) ✓
    '⎵runs' (0.628) 
    'run' (0.549) 


---

## Position Embedding Analysis

Let's analyze how position embeddings encode sequential information.


In [121]:
# Compare position embeddings
print("Position Embedding Similarity Matrix")
print("=" * 50)
print("\nComparing positions 0, 1, 2, 10, 100, 500, 1000:\n")

positions = [0, 1, 2, 10, 100, 500, 1000]
pos_embeddings = [wpe[p] for p in positions]

# Print header
print("     ", end="")
for p in positions:
    print(f"{p:>6}", end="")
print()

# Compute and print similarity matrix
for i, p1 in enumerate(positions):
    print(f"{p1:>4} ", end="")
    for j, p2 in enumerate(positions):
        sim = cosine_similarity(pos_embeddings[i], pos_embeddings[j])
        print(f"{sim:>6.2f}", end="")
    print()


Position Embedding Similarity Matrix

Comparing positions 0, 1, 2, 10, 100, 500, 1000:

          0     1     2    10   100   500  1000
   0   1.00  0.52  0.50  0.44  0.30  0.06 -0.22
   1   0.52  1.00  0.92  0.72  0.31 -0.07 -0.41
   2   0.50  0.92  1.00  0.83  0.34 -0.14 -0.47
  10   0.44  0.72  0.83  1.00  0.25 -0.21 -0.61
 100   0.30  0.31  0.34  0.25  1.00  0.31 -0.41
 500   0.06 -0.07 -0.14 -0.21  0.31  1.00  0.32
1000  -0.22 -0.41 -0.47 -0.61 -0.41  0.32  1.00


In [122]:
# How does position similarity decay with distance?
print("\nPosition Similarity vs Distance")
print("=" * 50)
print("\nHow similar is position 0 to positions at various distances?\n")

base_pos = 0
base_emb = wpe[base_pos]

distances = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]

print(f"Distance from position {base_pos}:")
for dist in distances:
    if dist < wpe.shape[0]:
        sim = cosine_similarity(base_emb, wpe[dist])
        bar = "█" * int((sim + 1) * 15)  # Shift to positive range for visualization
        print(f"  {dist:>4}: {sim:+.4f} {bar}")



Position Similarity vs Distance

How similar is position 0 to positions at various distances?

Distance from position 0:
     1: +0.5242 ██████████████████████
     2: +0.5021 ██████████████████████
     5: +0.4890 ██████████████████████
    10: +0.4443 █████████████████████
    20: +0.3869 ████████████████████
    50: +0.3645 ████████████████████
   100: +0.2991 ███████████████████
   200: +0.1377 █████████████████
   500: +0.0612 ███████████████
  1000: -0.2178 ███████████


---

## Same Word, Different Position = Different Representation

A key insight: the **same word at different positions** gets a **different final embedding**!

```
"The cat and the dog"  
      ^           ^
   "the" at      "the" at
   position 0    position 3
```

Even though both are the word "the", they'll have different input embeddings because position embeddings are added.


In [123]:
# Demonstrate: same token, different positions
# Note: "The" and " the" are DIFFERENT tokens (capitalization + space matters!)
# We need a sentence where the SAME token appears twice, like " the" twice
text = "I saw the dog and the cat"
token_ids_demo = tokenizer.encode(text)
tokens_demo = [tokenizer.get_token_string(tid) for tid in token_ids_demo]

print(f"Text: '{text}'")
print(f"\nTokens: {[t.replace('Ġ', '⎵') for t in tokens_demo]}")
print(f"Token IDs: {token_ids_demo}")

# Find repeated tokens
seen = {}
for i, tid in enumerate(token_ids_demo):
    if tid in seen:
        seen[tid].append(i)
    else:
        seen[tid] = [i]

# Find tokens that appear multiple times
repeated = {tid: positions for tid, positions in seen.items() if len(positions) > 1}

if repeated:
    print("\n" + "=" * 60)
    print("Same token at different positions:")
    print("=" * 60)
    
    for tid, positions in repeated.items():
        token_str = tokenizer.get_token_string(tid).replace("Ġ", "⎵")
        print(f"\nToken '{token_str}' (ID {tid}) appears at positions: {positions}")
        
        # Get token embedding (same for both)
        token_emb = wte[tid]
        
        # Get different position embeddings
        for pos in positions:
            pos_emb = wpe[pos]
            combined = token_emb + pos_emb
            print(f"  Position {pos}:")
            print(f"    Token emb[:3]:    [{', '.join(f'{v:.3f}' for v in token_emb[:3])}]")
            print(f"    Position emb[:3]: [{', '.join(f'{v:+.3f}' for v in pos_emb[:3])}]")
            print(f"    Combined[:3]:     [{', '.join(f'{v:.3f}' for v in combined[:3])}]")
        
        # Compare the combined embeddings
        emb1 = wte[tid] + wpe[positions[0]]
        emb2 = wte[tid] + wpe[positions[1]]
        sim = cosine_similarity(emb1, emb2)
        print(f"\n  Similarity between '{token_str}' at pos {positions[0]} vs {positions[1]}: {sim:.4f}")
        print(f"  (Not 1.0 because different position embeddings are added!)")


Text: 'I saw the dog and the cat'

Tokens: ['I', '⎵saw', '⎵the', '⎵dog', '⎵and', '⎵the', '⎵cat']
Token IDs: [40, 2497, 262, 3290, 290, 262, 3797]

Same token at different positions:

Token '⎵the' (ID 262) appears at positions: [2, 5]
  Position 2:
    Token emb[:3]:    [-0.039, 0.005, 0.042]
    Position emb[:3]: [+0.004, -0.085, +0.055]
    Combined[:3]:     [-0.035, -0.080, 0.097]
  Position 5:
    Token emb[:3]:    [-0.039, 0.005, 0.042]
    Position emb[:3]: [+0.010, -0.034, +0.131]
    Combined[:3]:     [-0.030, -0.029, 0.173]

  Similarity between '⎵the' at pos 2 vs 5: 0.9516
  (Not 1.0 because different position embeddings are added!)


---

## The Complete Embedding Function

Let's create a final, complete embedding function that we'll use in future notebooks.


In [124]:
class GPT2Embeddings:
    """
    GPT-2 Embedding layer.
    
    Combines token embeddings and position embeddings to create
    the input representation for the transformer.
    """
    
    def __init__(self, wte, wpe, tokenizer):
        """
        Args:
            wte: Token embedding matrix [vocab_size, n_embd]
            wpe: Position embedding matrix [n_ctx, n_embd]
            tokenizer: GPT2Tokenizer instance
        """
        self.wte = wte
        self.wpe = wpe
        self.tokenizer = tokenizer
        
        self.vocab_size = wte.shape[0]
        self.n_embd = wte.shape[1]
        self.n_ctx = wpe.shape[0]
    
    def __call__(self, text):
        """
        Embed text into transformer input.
        
        Args:
            text: Input string
            
        Returns:
            dict with:
                - embeddings: (seq_len, n_embd) input embeddings
                - token_ids: list of token IDs
                - tokens: list of token strings
        """
        # Tokenize
        token_ids = self.tokenizer.encode(text)
        seq_len = len(token_ids)
        
        if seq_len > self.n_ctx:
            raise ValueError(f"Sequence length {seq_len} exceeds max context {self.n_ctx}")
        
        # Look up embeddings
        token_emb = self.wte[token_ids]
        position_emb = self.wpe[:seq_len]
        
        # Combine
        embeddings = token_emb + position_emb
        
        # Token strings for debugging
        tokens = [self.tokenizer.get_token_string(tid) for tid in token_ids]
        
        return {
            "embeddings": embeddings,
            "token_ids": token_ids,
            "tokens": tokens,
            "seq_len": seq_len,
        }
    
    def embed_ids(self, token_ids):
        """
        Embed pre-tokenized IDs.
        
        Args:
            token_ids: List or array of token IDs
            
        Returns:
            (seq_len, n_embd) embeddings
        """
        seq_len = len(token_ids)
        token_emb = self.wte[token_ids]
        position_emb = self.wpe[:seq_len]
        return token_emb + position_emb
    
    def __repr__(self):
        return (f"GPT2Embeddings(vocab_size={self.vocab_size}, "
                f"n_embd={self.n_embd}, n_ctx={self.n_ctx})")


# Create our embedding layer
embed_layer = GPT2Embeddings(wte, wpe, tokenizer)
print(embed_layer)


GPT2Embeddings(vocab_size=50257, n_embd=768, n_ctx=1024)


In [125]:
# Test the complete embedding layer
test_texts = [
    "Hello, world!",
    "The quick brown fox jumps over the lazy dog.",
    "GPT-2 processes text using token and position embeddings.",
]

print("Testing GPT2Embeddings")
print("=" * 60)

for text in test_texts:
    result = embed_layer(text)
    print(f"\nText: '{text[:50]}{'...' if len(text) > 50 else ''}'")
    print(f"  Tokens: {result['seq_len']}")
    print(f"  Embedding shape: {result['embeddings'].shape}")
    print(f"  Token breakdown: {[t.replace('Ġ', '⎵') for t in result['tokens'][:5]]}...")


Testing GPT2Embeddings

Text: 'Hello, world!'
  Tokens: 4
  Embedding shape: (4, 768)
  Token breakdown: ['Hello', ',', '⎵world', '!']...

Text: 'The quick brown fox jumps over the lazy dog.'
  Tokens: 10
  Embedding shape: (10, 768)
  Token breakdown: ['The', '⎵quick', '⎵brown', '⎵fox', '⎵jumps']...

Text: 'GPT-2 processes text using token and position embe...'
  Tokens: 14
  Embedding shape: (14, 768)
  Token breakdown: ['G', 'PT', '-', '2', '⎵processes']...


---

## Embedding Statistics

Let's look at some statistics of the embeddings to understand their properties.


In [126]:
print("Embedding Statistics")
print("=" * 60)

print("\nToken Embeddings (wte):")
print(f"  Shape: {wte.shape}")
print(f"  Mean: {wte.mean():.6f}")
print(f"  Std:  {wte.std():.6f}")
print(f"  Min:  {wte.min():.6f}")
print(f"  Max:  {wte.max():.6f}")
print(f"  Avg L2 norm: {np.linalg.norm(wte, axis=1).mean():.4f}")

print("\nPosition Embeddings (wpe):")
print(f"  Shape: {wpe.shape}")
print(f"  Mean: {wpe.mean():.6f}")
print(f"  Std:  {wpe.std():.6f}")
print(f"  Min:  {wpe.min():.6f}")
print(f"  Max:  {wpe.max():.6f}")
print(f"  Avg L2 norm: {np.linalg.norm(wpe, axis=1).mean():.4f}")

# Compare magnitudes
token_norm = np.linalg.norm(wte, axis=1).mean()
pos_norm = np.linalg.norm(wpe, axis=1).mean()
print(f"\nRelative magnitude:")
print(f"  Token embedding avg norm: {token_norm:.4f}")
print(f"  Position embedding avg norm: {pos_norm:.4f}")
print(f"  Ratio (token/position): {token_norm/pos_norm:.2f}x")


Embedding Statistics

Token Embeddings (wte):
  Shape: (50257, 768)
  Mean: 0.000380
  Std:  0.143696
  Min:  -1.269817
  Max:  1.785156
  Avg L2 norm: 3.9585

Position Embeddings (wpe):
  Shape: (1024, 768)
  Mean: -0.000679
  Std:  0.122691
  Min:  -4.538114
  Max:  4.065311
  Avg L2 norm: 3.3901

Relative magnitude:
  Token embedding avg norm: 3.9585
  Position embedding avg norm: 3.3901
  Ratio (token/position): 1.17x


### What Does the Ratio Signify?

The **ratio of token to position embedding norms** (~1.17x) tells us about the **relative importance of semantic meaning vs. positional information** in the combined embedding.

Since the input to the transformer is simply:
```
input_embedding = token_embedding + position_embedding
```

If one were much larger than the other, it would **dominate** the combined signal, drowning out the smaller one.

**Interpretation:**
- **Ratio ≈ 1**: Both sources of information contribute roughly equally
- **Ratio >> 1**: Token identity would dominate over position
- **Ratio << 1**: Position would dominate over token meaning

**Why GPT-2's ~1.17x ratio is good:**

This near-balance indicates GPT-2 learned to give roughly equal weight to:
- **"What is this word?"** (semantic content from token embeddings)
- **"Where is this word?"** (sequential position from position embeddings)

This makes sense because both pieces of information are critical for language understanding — you need to know both *what* was said and *in what order* to understand meaning (e.g., "dog bites man" vs. "man bites dog").


---

## Summary: What We Learned

### The Embedding Pipeline

```
Input Text
    |
    v
+----------------+
|   Tokenizer    |  "Hello world" -> [15496, 995]
+-------+--------+
        |
        v
+----------------+
|  Token Embed   |  wte[15496], wte[995] -> (2, 768)
|  (wte lookup)  |
+-------+--------+
        |
        v
+----------------+
| Position Emb   |  wpe[0], wpe[1] -> (2, 768)
| (wpe lookup)   |
+-------+--------+
        |
        v
+----------------+
|      Add       |  token_emb + position_emb -> (2, 768)
+-------+--------+
        |
        v
  Input to Transformer (2, 768)
```

### Key Concepts

1. **Token Embeddings** (`wte.weight`)
   - Shape: `[50257, 768]` - one row per vocabulary token
   - Learned during training to capture semantic meaning
   - Similar words have similar embeddings

2. **Position Embeddings** (`wpe.weight`)
   - Shape: `[1024, 768]` - one row per possible position
   - Tell the model WHERE each token appears
   - GPT-2 uses learned (not sinusoidal) positions

3. **Combination**
   - Simple addition: `input = token_emb + position_emb`
   - Same word at different positions -> different representations

4. **Embedding Properties**
   - Cosine similarity captures semantic relationships
   - Vector arithmetic works: `king - man + woman = queen`
   - 768 dimensions encode diverse features

### Next Step

Now that we can convert text into meaningful vector representations, we're ready to learn about **attention** - the mechanism that allows the model to relate tokens to each other!
